In [1]:
from torch.utils.data.sampler import SubsetRandomSampler
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tensorboard_logger import configure, log_value
from model import RecurrentAttention
from torchvision import transforms
from torchvision import datasets
import torch.nn.functional as F
import matplotlib.pyplot as plt
from utils import AverageMeter
from config import get_config
from utils import plot_images
from trainer import Trainer
from tqdm import tqdm
import pandas as pd
import torchvision
import numpy as np
import random
import pickle
import shutil
import torch
import time

import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

from utils import *

In [2]:
q = 4
with open("../../pooling/data/migration_data.json") as m:
    mig_data = json.load(m)
m.close()
mig_data = pd.DataFrame.from_dict(mig_data, orient = 'index').reset_index()
mig_data.columns = ['muni_id', 'num_migrants']
mig_data['class'] = pd.qcut(mig_data['num_migrants'], q = q, labels = [i for i in range(q)])
mig_data.head()

,muni_id,num_migrants,class
0,484001001,42055.0,3
1,484001002,4017.0,3
2,484001003,11992.0,3
3,484001004,762.0,2
4,484001005,7551.0,3


In [3]:
image_names = get_png_names("../../attn/data/MEX/")
y_class, y_mig = get_ys(image_names, mig_data)

with open("../val_images.txt") as ims:
    val_names = ims.read().splitlines()
    
train_names = [i for i in image_names if i not in val_names]

val_indices = [image_names.index(i) for i in val_names]
train_indices = [image_names.index(i) for i in train_names]
train_num = int(len(train_indices))

In [4]:
# train_indices = train_indices[0:25]
# val_indices = val_indices[0:10]

In [5]:
batch_size = 1
train = [(torchvision.transforms.functional.adjust_brightness(load_inputs(image_names[i]), brightness_factor = 2).squeeze(), y_mig[i]) for i in train_indices]
val = [(torchvision.transforms.functional.adjust_brightness(load_inputs(image_names[i]), brightness_factor = 2).squeeze(), y_mig[i]) for i in val_indices]
train_dl = torch.utils.data.DataLoader(train, batch_size = batch_size, shuffle = True)
val_dl = torch.utils.data.DataLoader(val, batch_size = batch_size, shuffle = True)

In [6]:
print("Num training: ", len(train_dl))
print("Num validation: ", len(val_dl))

Num training:  1446
Num validation:  620


In [ ]:
config, unparsed = get_config()
trainer = Trainer(config, (train_dl, val_dl))
trainer.train()

0.1s - loss: 1988.110 - mae: 1988.109:   0%|          | 0/1446 [00:00<?, ?it/s]


[*] Train on 1446 samples, validate on 620 samples

Epoch: 1/200 - LR: 0.000300


65.7s - loss: 76.848 - mae: 75.911: 100%|██████████| 1446/1446 [01:05<00:00, 22.00it/s]      


train loss: 1872.591 - val loss: 1799.920- train mae: 1860.536 - val mae: 1641.404 [*]


0.2s - loss: 210.618 - mae: 210.556:   0%|          | 3/1446 [00:00<01:26, 16.75it/s]  

...

0.0s - loss: 811.079 - mae: 810.711:   0%|          | 0/1446 [00:00<?, ?it/s]

train loss: 1752.322 - val loss: 1533.989- train mae: 1721.796 - val mae: 1523.083

Epoch: 36/200 - LR: 0.000030


  0%|          | 0/1446 [00:00<?, ?it/s]

train loss: 2021.756 - val loss: 1577.459- train mae: 1722.527 - val mae: 1515.838

Epoch: 37/200 - LR: 0.000030


44.9s - loss: 2115.797 - mae: 2115.511:  74%|███████▎  | 1066/1446 [00:44<00:14, 25.56it/s]  